# Backpropagation from Scratch (NumPy)
**Objective:** Implement a Multilayer Perceptron (MLP) and Backpropagation from scratch to understand the chain rule and gradient descent updates.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set seed for reproducibility
np.random.seed(42)

## Problem Setup (Minimal)
We will build a simple **MLP** for binary classification:
1.  **Input Layer** ($X$)
2.  **Hidden Layer:** Linear transform ($W_1, b_1$) -> ReLU activation
3.  **Output Layer:** Linear transform ($W_2, b_2$) -> Sigmoid activation -> Prediction ($\hat{y}$)

**Backpropagation:** The method to compute gradients of the loss function with respect to weights using the **Chain Rule**.
$$ \frac{\partial L}{\partial W} = \frac{\partial L}{\partial \hat{y}} \cdot \frac{\partial \hat{y}}{\partial Z} \cdot \frac{\partial Z}{\partial W} $$

## Data

In [ ]:
# Generate Synthetic Binary Data (2 Gaussian Clusters)
n_samples = 400

# Cluster A (Class 0)
X_0 = np.random.randn(n_samples // 2, 2) + np.array([-2, -2])
y_0 = np.zeros((n_samples // 2, 1))

# Cluster B (Class 1)
X_1 = np.random.randn(n_samples // 2, 2) + np.array([2, 2])
y_1 = np.ones((n_samples // 2, 1))

# Combine and Shuffle
X = np.vstack([X_0, X_1])
y = np.vstack([y_0, y_1])

# Shuffle
indices = np.arange(n_samples)
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Train/Test Split (80/20)
split_idx = int(0.8 * n_samples)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# Visualize
plt.figure(figsize=(6, 5))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train.ravel(), cmap='bwr', alpha=0.6, edgecolors='k', label='Train')
plt.title("Synthetic Binary Classification Data")
plt.legend()
plt.show()

## Implementation (NumPy)

In [ ]:
def sigmoid(z):
    # Clip to avoid overflow/underflow for extreme values
    z = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def relu_deriv(z):
    return (z > 0).astype(float)

def init_params(d_in, d_hidden, d_out):
    """Initialize weights using He Initialization (good for ReLU)."""
    # He Init: scale by sqrt(2/fan_in)
    W1 = np.random.randn(d_in, d_hidden) * np.sqrt(2 / d_in)
    b1 = np.zeros((1, d_hidden))
    
    # Xavier Init for Sigmoid output (optional, but He is okay too)
    W2 = np.random.randn(d_hidden, d_out) * np.sqrt(1 / d_hidden)
    b2 = np.zeros((1, d_out))
    
    params = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return params

def forward(X, params):
    Z1 = X.dot(params["W1"]) + params["b1"]
    A1 = relu(Z1)
    
    Z2 = A1.dot(params["W2"]) + params["b2"]
    A2 = sigmoid(Z2)
    
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return cache, A2

def compute_loss(y, y_prob):
    """Binary Cross Entropy Loss."""
    m = y.shape[0]
    epsilon = 1e-15 # Prevent log(0)
    loss = -1/m * np.sum(y * np.log(y_prob + epsilon) + (1 - y) * np.log(1 - y_prob + epsilon))
    return loss

def backward(X, y, y_prob, cache, params):
    """Computes gradients."""
    m = X.shape[0]
    W2 = params["W2"]
    A1 = cache["A1"]
    Z1 = cache["Z1"]
    
    # Output layer error (dZ2 = A2 - Y for Sigmoid + CrossEntropy)
    dZ2 = y_prob - y
    
    # Gradients for Layer 2
    dW2 = (1/m) * A1.T.dot(dZ2)
    db2 = (1/m) * np.sum(dZ2, axis=0, keepdims=True)
    
    # Hidden layer error
    dA1 = dZ2.dot(W2.T)
    dZ1 = dA1 * relu_deriv(Z1)
    
    # Gradients for Layer 1
    dW1 = (1/m) * X.T.dot(dZ1)
    db1 = (1/m) * np.sum(dZ1, axis=0, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads

def update(params, grads, lr=0.1):
    """Update parameters using Gradient Descent."""
    params["W1"] -= lr * grads["dW1"]
    params["b1"] -= lr * grads["db1"]
    params["W2"] -= lr * grads["dW2"]
    params["b2"] -= lr * grads["db2"]
    return params

def predict(X, params, threshold=0.5):
    _, y_prob = forward(X, params)
    return (y_prob >= threshold).astype(int)

def accuracy(y, y_pred):
    return np.mean(y == y_pred)

## Gradient Checking (Validation)
Verifying our analytical gradients against numerical gradients computed via finite differences.
$$ \frac{dJ}{d\theta} \approx \frac{J(\theta + \epsilon) - J(\theta - \epsilon)}{2\epsilon} $$

In [ ]:
def check_gradients(X, y):
    """Checks gradients for W1 and W2."""
    # Init small random network
    params = init_params(d_in=2, d_hidden=5, d_out=1)
    epsilon = 1e-7
    
    # Get Analytical Gradients
    cache, y_prob = forward(X, params)
    grads = backward(X, y, y_prob, cache, params)
    
    # Check a few random elements in W1
    print("Checking Gradient for W1...")
    for _ in range(3):
        row, col = np.random.randint(0, params["W1"].shape[0]), np.random.randint(0, params["W1"].shape[1])
        
        # Store original value
        original_val = params["W1"][row, col]
        
        # J(theta + eps)
        params["W1"][row, col] = original_val + epsilon
        _, prob_plus = forward(X, params)
        loss_plus = compute_loss(y, prob_plus)
        
        # J(theta - eps)
        params["W1"][row, col] = original_val - epsilon
        _, prob_minus = forward(X, params)
        loss_minus = compute_loss(y, prob_minus)
        
        # Numerical Grad
        grad_numerical = (loss_plus - loss_minus) / (2 * epsilon)
        
        # Analytical Grad
        grad_analytical = grads["dW1"][row, col]
        
        # Relative Error
        rel_error = abs(grad_analytical - grad_numerical) / (abs(grad_analytical) + abs(grad_numerical) + 1e-8)
        
        print(f"  W1[{row},{col}]: Analytic={grad_analytical:.6f}, Numerical={grad_numerical:.6f}, RelError={rel_error:.2e}")
        
        # Restore
        params["W1"][row, col] = original_val
        
    print("Gradient check passed if Relative Error is small (< 1e-5).")

# Run check on a small subset of data
check_gradients(X_train[:5], y_train[:5])

## Training

In [ ]:
# Hyperparameters
epochs = 1000
learning_rate = 0.1
d_hidden = 10

# Initialize
params = init_params(d_in=2, d_hidden=d_hidden, d_out=1)
loss_history = []

# Training Loop
for epoch in range(epochs):
    # Forward
    cache, y_prob = forward(X_train, params)
    
    # Loss
    loss = compute_loss(y_train, y_prob)
    loss_history.append(loss)
    
    # Backward
    grads = backward(X_train, y_train, y_prob, cache, params)
    
    # Update
    params = update(params, grads, lr=learning_rate)
    
    if epoch % 100 == 0:
        y_pred = predict(X_train, params)
        acc = accuracy(y_train, y_pred)
        print(f"Epoch {epoch}: Loss = {loss:.4f}, Acc = {acc:.2f}")

# Final Evaluation
y_pred_test = predict(X_test, params)
test_acc = accuracy(y_test, y_pred_test)
print(f"\nFinal Test Accuracy: {test_acc:.2f}")

# Plot Loss
plt.figure(figsize=(8, 4))
plt.plot(loss_history)
plt.title("Training Loss Curve")
plt.xlabel("Epochs")
plt.ylabel("Binary Cross Entropy Loss")
plt.grid(True)
plt.show()

## Decision Boundary (2D)

In [ ]:
def plot_decision_boundary(X, y, params):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),
                         np.arange(y_min, y_max, 0.05))
    
    # Predict on whole grid
    grid_points = np.c_[xx.ravel(), yy.ravel()]
    _, probs = forward(grid_points, params)
    Z = probs.reshape(xx.shape)
    
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='bwr')
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), cmap='bwr', edgecolors='k')
    plt.title("MLP Decision Boundary after Training")
    plt.show()

plot_decision_boundary(X_test, y_test, params)

## Results & Takeaways
*   **Gradient Checking:** The low relative error (order of 1e-9 or less) confirms our chain rule implementation is mathematically correct. This is the gold standard for debugging neural nets.
*   **Initialization:** We used 'He Initialization', which is standard for ReLU networks to keep the variance of activations similar across layers, preventing vanishing/exploding gradients initially.
*   **Non-Linearity:** The decision boundary is curved (unlike Logistic Regression), showing the MLP successfully combined the ReLU neurons to separate the data.
*   **Convergence:** The loss decreased smoothly. If the learning rate were too high, we might see oscillations; too low, and it would stall.

## Next Steps
*   Now that we have the mechanics, we need better **Optimization Algorithms** than plain SGD.
*   [Go to Optimization Algorithms (Momentum, Adam)](./optimization-algorithms.ipynb)